### Model Converter

The model has been converted using tensorflow

The code to download the model and then convert the model from tensorflow to IR model is provided

In [7]:
import numpy as np

In [ ]:
!pwd

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03.tar.gz

In [ ]:
!mv "/home/aswin/Documents/Courses/Udacity/Intel-Edge/Work/EdgeApp/License_Plate_Recognition/SOLID-Project-Framework/notebooks/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz" \
"/home/aswin/Documents/Courses/Udacity/Intel-Edge/Work/EdgeApp/License_Plate_Recognition/SOLID-Project-Framework/models/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz"
!mv "/home/aswin/Documents/Courses/Udacity/Intel-Edge/Work/EdgeApp/License_Plate_Recognition/SOLID-Project-Framework/notebooks/ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03.tar.gz" \
"/home/aswin/Documents/Courses/Udacity/Intel-Edge/Work/EdgeApp/License_Plate_Recognition/SOLID-Project-Framework/models/ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03.tar.gz"
!tar -xzvf "/home/aswin/Documents/Courses/Udacity/Intel-Edge/Work/EdgeApp/License_Plate_Recognition/SOLID-Project-Framework/models/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz"
!tar -xzvf "/home/aswin/Documents/Courses/Udacity/Intel-Edge/Work/EdgeApp/License_Plate_Recognition/SOLID-Project-Framework/models/ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03.tar.gz"

In [ ]:
SAVED_MODEL = "/home/aswin/Documents/Courses/Udacity/Intel-Edge/Work/EdgeApp/License_Plate_Recognition/SOLID-Project-Framework/models/ssdlite_mobilenet_v2_coco_2018_05_09/saved_model"
!saved_model_cli show --dir \
"/home/aswin/Documents/Courses/Udacity/Intel-Edge/Work/EdgeApp/License_Plate_Recognition/SOLID-Project-Framework/models/ssdlite_mobilenet_v2_coco_2018_05_09/saved_model"\
--tag_set serve --signature_def serving_default

In [ ]:
with tf.Session() as sess:
    loaded = tf.saved_model.load(export_dir=SAVED_MODEL)
    print(list(loaded.signatures.keys()))  # ["serving_default"]

In [ ]:
import object_detection

!protoc --proto_path="/home/aswin/Documents/Courses/Udacity/Intel-Edge/models/research/" \
"/home/aswin/Documents/Courses/Udacity/Intel-Edge/models/research/object_detection/protos/*.proto" \
--python_out="/home/aswin/Documents/Courses/Udacity/Intel-Edge/models/research_py"

In [19]:
class TensorflowGraphDef(object):

    def __init__(self, model_filepath):

        # The file path of model
        self.model_filepath = model_filepath
        # Initialize the model
        self.load_graph(model_filepath = self.model_filepath)

    def load_graph(self, model_filepath):
        '''
        Lode trained model.
        '''
        print('Loading model...')
        self.graph = tf.Graph()
        self.sess = tf.InteractiveSession(graph = self.graph)

        with tf.gfile.GFile(model_filepath, 'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())

        print('Check out the input placeholders:')
        nodes = [n.name + ' => ' +  n.op for n in graph_def.node if n.op in ('Placeholder')]
        for node in nodes:
            print(node)
            
        file_input = "panda.jpeg"

        # Define input tensor
        self.input = tf.placeholder(np.float32, shape = [1, 224, 224, 3], name='input')
#         self.input = images_data(file_input)
        print(self.input)
        self.dropout_rate = tf.placeholder(tf.float32, shape = [], name = 'dropout_rate')

        tf.import_graph_def(graph_def, {'input': self.input, 'dropout_rate': self.dropout_rate})

        print('Model loading complete!')

        '''
        # Get layer names
        layers = [op.name for op in self.graph.get_operations()]
        for layer in layers:
            print(layer)
        '''

        '''
        # Check out the weights of the nodes
        weight_nodes = [n for n in graph_def.node if n.op == 'Const']
        for n in weight_nodes:
            print("Name of the node - %s" % n.name)
            print("Value - " )
            print(tensor_util.MakeNdarray(n.attr['value'].tensor))
        '''

    def test(self, data):

        # Know your output node name
        output_tensor = self.graph.get_tensor_by_name("import/cnn/output:0")
        output = self.sess.run(output_tensor, feed_dict = {self.input: data, self.dropout_rate: 0})

        return output
    
def images_data(file_input):
    
    image = tf.image.decode_jpeg(tf.read_file(file_input))

    images = tf.expand_dims(image, 0)
    images = tf.cast(images, tf.float32) / 128.  - 1
    images.set_shape((None, None, None, 3))
    images = tf.image.resize_images(images, (224, 224))
    
    return images

def test_from_frozen_graph(model_filepath):

#     tf.reset_default_graph()

    # Load CIFAR10 dataset
    file_input = "panda.jpeg"

    model = TensorflowGraphDef(model_filepath = model_filepath)

    test_prediction_onehot = model.test(data = images_data(file_input))
    test_prediction = np.argmax(test_prediction_onehot, axis = 1).reshape((-1,1))
    test_accuracy = model_accuracy(label = y_test, prediction = test_prediction)

    print('Test Accuracy: %f' % test_accuracy)

In [22]:
import tensorflow as tf
from nets.mobilenet import mobilenet_v2

# tf.reset_default_graph()

# For simplicity we just decode jpeg inside tensorflow.
# But one can provide any input obviously.
file_input = tf.placeholder(tf.string, ())

image = tf.image.decode_jpeg(tf.read_file(file_input))

images = tf.expand_dims(image, 0)
images = tf.cast(images, tf.float32) / 128.  - 1
images.set_shape((None, None, None, 3))
images = tf.image.resize_images(images, (224, 224))

# Note: arg_scope is optional for inference.
with tf.contrib.slim.arg_scope(mobilenet_v2.training_scope(is_training=False)):
  logits, endpoints = mobilenet_v2.mobilenet(images)

In [20]:
test_from_frozen_graph("/home/aswin/Documents/Courses/Udacity/Intel-Edge/Work/EdgeApp/License_Plate_Recognition/SOLID-Project-Framework/models/ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03/frozen_inference_graph.pb")

Loading model...
Check out the input placeholders:
image_tensor => Placeholder
Tensor("input:0", shape=(1, 224, 224, 3), dtype=float32)


ValueError: Attempted to map inputs that were not found in graph_def: [dropout_rate:0, input:0]

In [ ]:
!protoc --proto_path="/home/aswin/Documents/Courses/Udacity/Intel-Edge/models/research/object_detection/protos/*.proto" --python_out=/home/aswin/Documents/Courses/Udacity/Intel-Edge/models/research/object_detection/

In [ ]:
!/usr/bin/python3.6 /opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py --mean_values="[127,127,127]" --scale="2.0" --saved_model_dir /home/aswin/Documents/Courses/Udacity/Intel-Edge/Work/EdgeApp/License_Plate_Recognition/SOLID-Project-Framework/models/ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03/saved_model/ --reverse_input_channels --input_shape=[1,224,224,3] --tensorflow_use_custom_operations_config /opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/ssd_v2_support.json --tensorflow_object_detection_api_pipeline_config /home/aswin/Documents/Courses/Udacity/Intel-Edge/Work/EdgeApp/License_Plate_Recognition/SOLID-Project-Framework/models/ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03/pipeline.config

In [ ]:
!/usr/bin/python3.6 /opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py --mean_values="[127,127,127]" --scale="2.0" --saved_model_dir /home/aswin/Documents/Courses/Udacity/Intel-Edge/Work/EdgeApp/License_Plate_Recognition/SOLID-Project-Framework/models/ssdlite_mobilenet_v2_coco_2018_05_09/saved_model/ --reverse_input_channels --input_shape=[1,224,224,3] --tensorflow_use_custom_operations_config /opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/ssd_v2_support.json --tensorflow_object_detection_api_pipeline_config /home/aswin/Documents/Courses/Udacity/Intel-Edge/Work/EdgeApp/License_Plate_Recognition/SOLID-Project-Framework/models/ssdlite_mobilenet_v2_coco_2018_05_09/pipeline.config

In [ ]:
!/usr/bin/python3.6 /opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py --input_model /home/aswin/Documents/Courses/Udacity/Intel-Edge/Work/EdgeApp/License_Plate_Recognition/SOLID-Project-Framework/models/ssdlite_mobilenet_v2_coco_2018_05_09/frozen_inference_graph.pb --tensorflow_object_detection_api_pipeline_config /home/aswin/Documents/Courses/Udacity/Intel-Edge/Work/EdgeApp/License_Plate_Recognition/SOLID-Project-Framework/models/ssdlite_mobilenet_v2_coco_2018_05_09/pipeline.config --reverse_input_channels --tensorflow_use_custom_operations_config /opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/ssd_v2_support.json

In [ ]:
!/usr/bin/python3.6 /opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py --input_model /home/aswin/Documents/Courses/Udacity/Intel-Edge/Work/EdgeApp/License_Plate_Recognition/SOLID-Project-Framework/models/ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03/frozen_inference_graph.pb --tensorflow_object_detection_api_pipeline_config /home/aswin/Documents/Courses/Udacity/Intel-Edge/Work/EdgeApp/License_Plate_Recognition/SOLID-Project-Framework/models/ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03/pipeline.config --reverse_input_channels --tensorflow_use_custom_operations_config /opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/ssd_v2_support.json